# TRANSACTIONS PROCESSOR

In Rewards-R-Tokens, the transactions processor is primarily responsible for:

1. Mocking credit card transactions
2. Calculate appropriate rewards token count
3. Interact with deployed smart contract
4. Transfer token to the customer address.

A credit card transaction in our blockchain world carries the following schema:

```code=json
    {
    transaction_id: int,
    customer_id: address_from_ganache,
    transaction_amount: int(in dollars),
    transaction_category: ['gas', 'restaurant', 'grocery', 'subscription', 'entertainment'],
    transaction_date: Date
    }
```

In [1]:
# Project Imports

from time import sleep
from web3 import Web3, Account
from dotenv import load_dotenv
from random import choice
from datetime import datetime

import os
import json
import random

In [2]:
# Declare local Ganache HTTP provider
local_ganache_http_provider = 'HTTP://127.0.0.1:7545'

# Connect to the local Ganache HTTP provider
w3=Web3(Web3.HTTPProvider(local_ganache_http_provider))

In [3]:
"""
!!! ATTENTION !!! ATTENTION !!! ATTENTION !!!

.env file must be present in the current working directory
wuth the following key:value pairs:

CONTRACT_ADDRESS=<Deployed Contact Address from Remix>
CONTRACT_OWNER_ADDRESS=<Contract Owner Address from Ganache>
CONTRACT_OWNER_PVT_KEY=<Private key of CONTRACT_OWNER_ADDRESS from Ganache>

You should pick the address of account that was used to deploy the contract 
as CONTRACT_OWNER_ADDRESS from Ganache.
"""


load_dotenv()
contract_address = os.getenv('CONTRACT_ADDRESS')
contract_owner_address = os.getenv('CONTRACT_OWNER_ADDRESS')
contract_owner_pvt_key = os.getenv('CONTRACT_OWNER_PVT_KEY')
print(f'Contract Deployed at: {contract_address} by {contract_owner_address}')

Contract Deployed at: 0x325dC9c17E415E9485324017EC19622cF55F78e2 by 0x74F0C22756370a259A4F6738673588E22B18BBDE


In [4]:
# Load Application Binary Interface (abi) file as JSON

with open('contract_abi.txt') as contract_abi_file:
    contract_abi_json = json.load(contract_abi_file)

# Connect to the deployed contract
rt_contract = w3.eth.contract(contract_address, abi=contract_abi_json)

In [5]:
# Get all accounts from local Ganache provider
accounts = w3.eth.accounts

# Remove contract owner address from accounts as we
# DO NOT want to use the owner address for transacations.
accounts.remove(contract_owner_address)

In [6]:
# Create a dictionary for total customer spendings
customer_totals = {}

In [7]:
# Declare of list of transaction categories
txn_categories = ['gas', 'restaurant',  'grocery', 'subscription', 'entertainment']

In [8]:
# Declare custom token conversion values for categories.
# TODO: This is a static list but this would be modified to be served via an API
#  to make it dynamic in future.

txn_categories_conversion_rate = { 
                                  'gas': 2.0,
                                  'restaurant': 1.5, 
                                  'grocery': 1.0,
                                  'subscription': 3.0,
                                  'entertainment': 2.5
                                 }

# Number of tokens awarded = transacation_amount * txn_categories_conversion_rate[transacation_category]
# This basically means the following:
# If a customer spends $10 on gas, they would get 20 tokens
# =====> $10 (transacation_amount) * 2.0 (txn_categories_conversion_rate['gas']))
# If a customer spends $9.99 on subscriptions, they would get 29.97 tokens 
# =====> $9.99 (transacation_amount) * 3.0 (txn_categories_conversion_rate['subscription']))

In [9]:
# Define a generator function to mock credit card transactions:

def credit_card_transactions(num_transacations=10):
    for i in range(num_transacations):
        tx = {}
        tx['id'] = i
        tx['customer_id'] = choice(accounts)
        tx['amount'] = round(random.uniform(0.5, 99.5), 2)
        tx['category'] = choice(txn_categories)
        tx['date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        yield tx
    

In [14]:
for c_txn in credit_card_transactions(50):
    num_tokens_award = c_txn['amount'] * txn_categories_conversion_rate[c_txn['category']]
    num_tokens_award = int(num_tokens_award) * (10**18)
    txn = {
    'from': contract_owner_address,
    'nonce': w3.eth.get_transaction_count(contract_owner_address),
    'gasPrice': w3.eth.gas_price
    }
    
    raw_txn = rt_contract.functions.mint(c_txn['customer_id'], num_tokens_award).buildTransaction(txn)
    
    signed_txn = w3.eth.account.signTransaction(raw_txn, private_key=contract_owner_pvt_key)
    
    txn_response = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    
    print(f"{num_tokens_award/(10**18)} tokens awarded to {c_txn['customer_id']} for spending ${c_txn['amount']} on {c_txn['category']}.")
    
    sleep(1.0)

17.0 tokens awarded to 0x594a6B067B737446Afdc488792C32942835725c3 for spending $8.72 on gas.
39.0 tokens awarded to 0x594a6B067B737446Afdc488792C32942835725c3 for spending $39.97 on grocery.
18.0 tokens awarded to 0xc4835F571731965E7154063080b8B825a5dd1394 for spending $18.19 on grocery.
30.0 tokens awarded to 0x594a6B067B737446Afdc488792C32942835725c3 for spending $30.42 on grocery.
92.0 tokens awarded to 0xD520bFC9Ff319A844F825Def63aFB334271876d2 for spending $61.86 on restaurant.
28.0 tokens awarded to 0xD520bFC9Ff319A844F825Def63aFB334271876d2 for spending $14.16 on gas.
95.0 tokens awarded to 0x664FD2fc5e6c478c2bf464976e25C18dc1881242 for spending $38.24 on entertainment.
213.0 tokens awarded to 0x9dDed0Ab44D334f08E9B196534C35892074666fa for spending $71.23 on subscription.
74.0 tokens awarded to 0xb86A04f0F072f942073B83ad373E3565Aa0cfcaf for spending $29.8 on entertainment.
71.0 tokens awarded to 0xc4835F571731965E7154063080b8B825a5dd1394 for spending $47.94 on restaurant.
2.0 to

In [11]:
def get_account_balance(account_id):
    return rt_contract.functions.balanceOf(account_id).call();

In [12]:
print('Account Balance')
print('===============')

for account_no in accounts:
    print(f"{account_no}: {get_account_balance(account_no)}")

Account Balance
0xD520bFC9Ff319A844F825Def63aFB334271876d2: 846000000000000000000
0xc4835F571731965E7154063080b8B825a5dd1394: 687000000000000000000
0x743deaf34D734D37c7BCFC5b4FE5b18E0617e23F: 337000000000000000000
0x41971A0b1B4b928a7654bB5847Fc97704E95ce4b: 538000000000000000000
0x664FD2fc5e6c478c2bf464976e25C18dc1881242: 315000000000000000000
0xb86A04f0F072f942073B83ad373E3565Aa0cfcaf: 1178000000000000000000
0xB4230229eDF2a33eD67c0DDDf94B14a2fDaE4B9A: 471000000000000000000
0x594a6B067B737446Afdc488792C32942835725c3: 149000000000000000000
0x9dDed0Ab44D334f08E9B196534C35892074666fa: 472000000000000000000
